__[Open and try this file online (Colab)](https://colab.research.google.com/github/djekra/wiktionary_wortschatz/blob/master/1_wiktionary_parsen/x20_d00_mwparserfromhell.ipynb)__

# mwparserfromhell
* erstellt die Dateien wiktionary_merkmal.pickle und wiktionary_merkmal_text.pickle in data_00
* parst dafür wiktionary.wikitext mit mwparserfromhell
* File wiktionary wird nicht neu gespeichert

## Import

In [1]:
# blab init
try:
    import blab
except ImportError as e:
    !pip install blab
    import blab    
startup_notebook = blab.blab_startup()
%run $startup_notebook 

blab init
environment['dropbox_path'] = /home/me/Data_Linux/Dropbox
environment['lib_path']     = /home/me/Data_Linux/Dropbox/31_Projekte/01_Python/libs
Start Time: 17:51:44


time: 409 ms


In [ ]:
# alle anderen
try:
    import pandasklar as pak, bj_nlp, mwparserfromhell, pandarallel

except ImportError as e:
    !pip install pandasklar bj_nlp mwparserfromhell pandarallel
    import pandasklar as pak, bj_nlp, mwparserfromhell, pandarallel
    
grid       = pak.grid
check_mask = pak.check_mask      

## Einstellungen

In [2]:

version = '_all'
#version = '_short'
speicher_freigeben = True    # sollen alte Ergebnisse gelöscht werden

# Load
verzeichnis_load                 = 'data_00'
wiktionary_filename              = verzeichnis_load + '/wiktionary'               + version + '.pickle'  
wiktionary_trash_filename        = verzeichnis_load + '/wiktionary_trash'         + version + '.pickle'

# Save
verzeichnis_save                 = 'data_00'
wiktionary_merkmal_filename      = verzeichnis_save + '/wiktionary_merkmal'      + version + '.pickle'  
wiktionary_merkmal_text_filename = verzeichnis_save + '/wiktionary_merkmal_text' + version + '.pickle'  

# Was debuggen?
suche_debug = ['entlassen','missraten']

time: 13.7 ms


## Vorbereiten

In [54]:
# Verzeichnisse erzeugen
os.makedirs(verzeichnis_load, exist_ok=True) 
os.makedirs(verzeichnis_save, exist_ok=True) 

time: 48.7 ms (started: 2023-02-15 16:00:28 +00:00)


In [ ]:
# download additional source files
import os
if not os.path.exists(verzeichnis_load):
   !git clone https://github.com/djekra/wiktionary_wortschatz.git
   !cp -r wiktionary_wortschatz/1_wiktionary_parsen/Steuertabellen Steuertabellen
   !cp -r wiktionary_wortschatz/1_wiktionary_parsen/*.py .
#   !rm -r wiktionary_wortschatz

In [105]:
# Lade Verzeichnis data_00 von Google Drive
# Dieses Verzeichnis wurde in der vorherigen Verarbeitungsstufe angelegt (Notebook x10_d00_wiktionary_de_parser)
if environment['in_colab']:
    from google.colab import drive
    drive.mount('/content/drive') 
    !cd /content
    !cp -av drive/MyDrive/{verzeichnis_load} {verzeichnis_load}

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
'data_00' -> 'drive/MyDrive/data_00/data_00'
'data_00/dewiktionary-latest-pages-articles-multistream.xml.bz2' -> 'drive/MyDrive/data_00/data_00/dewiktionary-latest-pages-articles-multistream.xml.bz2'
'data_00/wiktionary_short.pickle' -> 'drive/MyDrive/data_00/data_00/wiktionary_short.pickle'
'data_00/wiktionary_trash_short.pickle' -> 'drive/MyDrive/data_00/data_00/wiktionary_trash_short.pickle'
'data_00/wiktionary_all.pickle' -> 'drive/MyDrive/data_00/data_00/wiktionary_all.pickle'
'data_00/wiktionary_trash_all.pickle' -> 'drive/MyDrive/data_00/data_00/wiktionary_trash_all.pickle'
time: 6.9 s (started: 2023-02-15 16:36:47 +00:00)


In [4]:
# Steuertabellen
from Steuertabellen.s01_Steuertabellen   import *

# mwparserfromhell eigene Ergänzungen 
from x24_process_wikicode import *
from x22_my_node          import *

INFO: Pandarallel will run on 2 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
time: 1.8 s


In [4]:
from pandarallel import pandarallel
nb_workers = os.cpu_count()
if nb_workers > 2:
    nb_workers = int((nb_workers / 2) - 1)
pandarallel.initialize(use_memory_fs=True, nb_workers=nb_workers, progress_bar=True)  

INFO: Pandarallel will run on 5 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
time: 28.5 ms


In [5]:
# Visualisiert einen Wikicode
def see(wikicode):
    return wikicode._get_tree(wikicode,list(),True,0)    

time: 42.1 ms


## Laden

In [6]:
# Laden
wiktionary       = pak.load_pickle(wiktionary_filename)
wiktionary_trash = pak.load_pickle(wiktionary_trash_filename)

time: 10.3 s


In [7]:
print('Einträge:', wiktionary.shape[0])

Einträge: 806916
time: 31.6 ms


In [8]:
# wiktionary anschauen
mask = wiktionary.title.isin(suche_debug) 
r = wiktionary[mask].sort_values(['title','section_id2'])
grid(r)

2 rows


time: 294 ms


In [9]:
pak.analyse_cols(wiktionary)

,col_name,datatype_instance,datatype,datatype_short,datatype_suggest,is_numeric,is_string,is_hashable,nan_allowed,mem_usage,ntypes,nunique,nnan,ndups,n,vmin,vmean,vmedian,vmax,vsum
0,__index__,int64,np.int64,int64,np.int32,True,False,True,False,80.0 B,1,806916,0,0,806916,0.0,403457.50,403457.5,806915.0,3.255563e+11
1,section_id,int32,pd.Int32,Int32,,True,False,True,True,50.0 B,1,806916,0,0,806916,0.0,60332729.62,59486750.0,107835100.0,4.868344e+13
2,page_id,int32,pd.Int32,Int32,,True,False,True,True,50.0 B,1,772285,0,34631,806916,0.0,603327.30,594867.5,1078351.0,4.868344e+11
3,page_part,int8,pd.Int8,Int8,,True,False,True,True,20.0 B,1,6,0,806910,806916,0.0,0.05,0.0,5.0,3.654000e+04
4,section_id2,str,pd.string,string,,False,True,True,True,733.0 B,1,806916,0,0,806916,&c.,NaN,NaN,μm,NaN
5,title,str,pd.string,string,,False,True,True,True,733.0 B,1,772284,0,34632,806916,&c.,NaN,NaN,μm,NaN
6,lemma,str,pd.string,string,,False,True,True,True,677.0 B,1,153738,0,653178,806916,&c.,NaN,NaN,μm,NaN
7,inflected,bool_,bool,bool,,True,False,True,True,10.0 B,1,2,0,806914,806916,0.0,0.83,NaN,1.0,6.717990e+05
8,pos,dict,object,object,,False,True,False,True,2.2 KB,1,165,0,806751,806916,NaN,NaN,NaN,NaN,NaN
9,rhymes,list,object,object,,False,True,False,True,464.0 B,1,38307,503744,264865,806916,[a],NaN,NaN,"[ʏʃəs, yːʃəs]",NaN


time: 14.6 s


In [10]:
pak.memory_consumption(locals())

,name,rtype,size
0,wiktionary,"(DataFrame, Series)",2.8 GB
1,wiktionary_trash,"(DataFrame, Series)",15.3 MB
2,mask,"(Series, bool)",788.1 KB
3,translate_tag,"(DataFrame, Series)",37.1 KB
4,translate_tagZ,"(DataFrame, Series)",20.9 KB
5,r,"(DataFrame, Series)",17.8 KB
6,_,"(DataFrame, Series)",15.8 KB
7,_9,"(DataFrame, Series)",15.8 KB
8,_i2,"(str,)",1.5 KB
9,Stop,"(type,)",1.0 KB


time: 1.04 s


## Steuertabelle plan_merkmal

In [12]:
blab.help(plan_merkmal_erstellen)

* definiert in Steuertabellen/s01_Steuertabellen.py
* `merkmal`: Name des Merkmals
* `template`: Name des Mediawiki-Templates, das diese Information enthält
* `collect`: Extractionsmethode
   * L extrahiert nur Links 
   * T extrahiert komplette Text-Absätze
   * Ü extrahiert Übersetzungen
   * P extrahiert Template-Parameter (Substring reicht)
   * E checkt nur auf Existenz
* `plan`:
   * 1: die Merkmale, die in wiktionary_merkmal gespeichert werden
   * 2: die Merkmale, die in wiktionary_merkmal_text gespeichert werden   
   * 3: Reihenfolge und Liste aller Merkmale aus flexion / übersicht <br>
        sowie die Merkmale aus named_entities und lexeme_manuell
* `sort`: Irgendwas zum Sortieren der Merkmale
* `is_lex`: Ist das Merkmal ein Lexem?    

time: 48.3 ms


In [13]:
plan_merkmal = plan_merkmal_erstellen()
grid(plan_merkmal)

102 rows


time: 136 ms


## ausprobieren

In [14]:
# Eine Testzeile aus wiktionary (als Series)
mask = (wiktionary.section_id2 == 'Haus')
testzeile = wiktionary[mask].iloc[0]
wikicode = mwparserfromhell.parse(testzeile.wikitext)

# testzeile 
# testzeile.wikitext
# wikicode

time: 484 ms


In [15]:
# extract_parameters ausprobieren (funktioniert z.B. mit 'kein')
r = extract_parameters(testzeile, 'Pronomina-Tabelle', 'übersicht', wikicode=wikicode )
grid(r, color=None)

No rows
time: 53.9 ms


In [16]:
#r.node_debug = r.node_debug.astype('str')
pak.analyse_cols(r)

,col_name,datatype_instance,datatype,datatype_short,datatype_suggest,is_numeric,is_string,is_hashable,nan_allowed,mem_usage,ntypes,nunique,nnan,ndups,n,vmin,vmean,vmedian,vmax,vsum
0,__index__,,object,object,,False,True,True,True,0.0 B,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN


time: 79.5 ms


In [17]:
# extract ausprobieren
df = extract(testzeile, 'Unterbegriffe', 'down', wikicode=wikicode)
grid(df)

408 rows


time: 137 ms


In [18]:
# extract_all ausprobieren
mask = plan_merkmal.plan == 2
plan = plan_merkmal[mask]
df = extract_all(testzeile, plan) 
df

,node_num,node_meta,node_link,node_text,node_kontext,node_debug,section_id,section_id2,merkmal
0,[1],,,zu einem bestimmten Zweck erbautes Gebäude,,,21600,Haus,def
1,[2],,,zum Wohnen dienendes und genutztes Gebäude,,,21600,Haus,def
2,[3],,,"aus mehreren Räumen bestehender, abgetrennter ...",,,21600,Haus,def
3,[4],umgangssprachlich,,umgangssprachlich Gesamtheit der Bewohner in ...,,,21600,Haus,def
4,[5],,,"Gesamtheit der Personen, die sich in einem bes...",,,21600,Haus,def
...,...,...,...,...,...,...,...,...,...
5,"[2, 3]",,,mein Haus ist meine Burg,,,21600,Haus,bsp_sp
6,"[2, 3, 7, 9]",,,ein Haus kann nicht zwei Hunde nähren,,,21600,Haus,bsp_sp
7,[3],,,"lässt du einen ins Haus kommen, er kommt dir b...",,,21600,Haus,bsp_sp
8,[3],,,"Nord, Ost, Süd, West, zu Haus ist’s am best",,,21600,Haus,bsp_sp


time: 405 ms


In [ ]:
raise Stop

## wiktionary_merkmal_text

In [19]:
# plan
mask = plan_merkmal.plan == 2
plan = plan_merkmal[mask]
plan

,merkmal,template,collect,plan,sort,is_lex
38,def,Bedeutungen,T,2,,False
39,herkunft,Herkunft,T,2,,False
40,bsp,Beispiele,T,2,,False
41,bsp_re,Redewendungen,T,2,,False
42,bsp_gw,Geflügelte Worte,T,2,,False
43,bsp_sp,Sprichwörter,T,2,,False


time: 60.2 ms


In [20]:
# all: 48 min
# 100K: 3 min
# apply erzeugt eine Series von DataFrames
# tolist macht daraus eine Liste
# und concat macht aus der Liste von DataFrames ein Gesamt-DataFrame
wiktionary_merkmal_text = pd.concat(  wiktionary.parallel_apply(extract_all, plan=plan, axis=1).tolist()  )

Executing shutdown due to inactivity...


2022-12-23 00:22:50,119 - INFO     - Executing shutdown due to inactivity...


Executing shutdown...


2022-12-23 00:22:51,709 - INFO     - Executing shutdown...


Exception on /shutdown [GET]
Traceback (most recent call last):
  File "/home/me/mambaforge/envs/python_06/lib/python3.9/site-packages/flask/app.py", line 2525, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/me/mambaforge/envs/python_06/lib/python3.9/site-packages/flask/app.py", line 1822, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/me/mambaforge/envs/python_06/lib/python3.9/site-packages/flask/app.py", line 1820, in full_dispatch_request
    rv = self.dispatch_request()
  File "/home/me/mambaforge/envs/python_06/lib/python3.9/site-packages/flask/app.py", line 1796, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  File "/home/me/mambaforge/envs/python_06/lib/python3.9/site-packages/dtale/app.py", line 435, in shutdown
    shutdown_server()
  File "/home/me/mambaforge/envs/python_06/lib/python3.9/site-packages/dtale/app.py", line 421, in shutdown_server
    raise RuntimeError("Not 

2022-12-23 00:22:51,757 - ERROR    - Exception on /shutdown [GET]
Traceback (most recent call last):
  File "/home/me/mambaforge/envs/python_06/lib/python3.9/site-packages/flask/app.py", line 2525, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/me/mambaforge/envs/python_06/lib/python3.9/site-packages/flask/app.py", line 1822, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/me/mambaforge/envs/python_06/lib/python3.9/site-packages/flask/app.py", line 1820, in full_dispatch_request
    rv = self.dispatch_request()
  File "/home/me/mambaforge/envs/python_06/lib/python3.9/site-packages/flask/app.py", line 1796, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  File "/home/me/mambaforge/envs/python_06/lib/python3.9/site-packages/dtale/app.py", line 435, in shutdown
    shutdown_server()
  File "/home/me/mambaforge/envs/python_06/lib/python3.9/site-packages/dtale/app.py", line 421, in shutdow

time: 1h 53min 8s


In [21]:
# nachbearbeiten
wiktionary_merkmal_text = pak.reset_index( wiktionary_merkmal_text )
wiktionary_merkmal_text = pak.move_cols(   wiktionary_merkmal_text, ['section_id','section_id2','merkmal'])
wiktionary_merkmal_text = pak.drop_cols(   wiktionary_merkmal_text, ['node_link','node_kontext','node_debug'])
wiktionary_merkmal_text = pak.rename_col(  wiktionary_merkmal_text, 'node_num',  'num')
wiktionary_merkmal_text = pak.rename_col(  wiktionary_merkmal_text, 'node_meta', 'meta')
wiktionary_merkmal_text = pak.rename_col(  wiktionary_merkmal_text, 'node_text', 'data')    

time: 1.54 s


## Kontrolle: wiktionary_merkmal_text

In [22]:
# Merkmale Häufigkeit
v = pak.analyse_freqs(wiktionary_merkmal_text, 'merkmal') 
v

,merkmal,merkmal_count,merkmal_percent,graph
0,bsp,332645,53.7,##########################
1,def,181880,29.3,##############
2,herkunft,98483,15.9,#######
3,bsp_re,6054,1.0,
4,bsp_sp,628,0.1,
5,bsp_gw,104,0.0,


time: 267 ms


In [23]:
# plan erfüllt?
geplant = set(plan.merkmal)
done    = set(v.merkmal)
nicht_erfüllt = geplant - done
print(nicht_erfüllt)
assert len(nicht_erfüllt) == 0

set()
time: 38.3 ms


In [24]:
# Sichtkontrolle
mask = wiktionary.section_id2.isin(suche_debug)
suche_section_ids = wiktionary[mask].section_id.tolist()
mask = wiktionary_merkmal_text.section_id.isin(suche_section_ids)
wiktionary_merkmal_text[mask]

,section_id,section_id2,merkmal,num,meta,data
81425,2256800,entlassen,def,[1],transitiv,transitiv jemandem erlauben zu gehen
81426,2256800,entlassen,def,[2],transitiv,transitiv jemandem die Arbeitsstelle kündigen
81427,2256800,entlassen,herkunft,[],etymologisch,etymologisch von mittelhochdeutsch entlāʒe...
81428,2256800,entlassen,bsp,[1],entlassen,Die Schüler wurden für heute entlassen.
81429,2256800,entlassen,bsp,[1],entlassen,Er wurde aus dem Gefängnis entlassen.
81430,2256800,entlassen,bsp,[1],entlassen,„Als er nach rund drei Monaten [Anmerkung: aus...
81431,2256800,entlassen,bsp,[2],entlassen,Sie wurde von der Firma entlassen.
294077,24215000,missraten,def,[1],,nicht gelingen
294078,24215000,missraten,bsp,[1],missraten,Dieser Kuchen ist vollkommen missraten.


time: 1.17 s


## Speichern

In [25]:
# Speichern   
if wiktionary_merkmal_text_filename:
    pak.dump_pickle(wiktionary_merkmal_text, wiktionary_merkmal_text_filename)   

time: 5.43 s


In [26]:
# Speicher
if speicher_freigeben:
    try:
        del wiktionary_merkmal_text
    except:
        pass
pak.memory_consumption(locals())

,name,rtype,size
0,wiktionary,"(DataFrame, Series)",2.8 GB
1,wiktionary_trash,"(DataFrame, Series)",15.3 MB
2,mask,"(Series, bool)",605.4 KB
3,testzeile,"(Series, int32)",344.8 KB
4,df,"(DataFrame, Series)",91.3 KB
5,_18,"(DataFrame, Series)",91.3 KB
6,translate_tag,"(DataFrame, Series)",37.1 KB
7,plan_merkmal,"(DataFrame, Series)",26.7 KB
8,translate_tagZ,"(DataFrame, Series)",20.9 KB
9,_9,"(DataFrame, Series)",15.8 KB


time: 21.3 s


In [27]:
#pak.analyse_cols(wiktionary)

time: 35.7 ms


## wiktionary_merkmal

In [28]:
# plan
mask = plan_merkmal.plan == 1
plan = plan_merkmal[mask]
plan

,merkmal,template,collect,plan,sort,is_lex
0,noSteig,kSt.,E,1,,False
1,noSing,kSg.,E,1,,False
2,nurWortart,Wortart fehlt,P,1,,False
3,istName,Navigationsleiste Anthroponyme,E,1,,False
4,Wikispecies,Wikispecies,E,1,,False
5,Wikivoyage,Wikivoyage,E,1,,False
6,Taxonomie,Artikel Biologische Taxonomie,E,1,,False
7,Präfix,Präfixe (Deutsch),E,1,,False
8,Suffix,Suffixe (Deutsch),E,1,,False
9,VMaßeinheiten,Vorsätze für Maßeinheiten,E,1,,False


time: 48.2 ms


In [29]:
# all: 2h 22min
# 100K: 8min 44s
# ERROR1 2ERROR
# apply erzeugt eine Series von DataFrames
# tolist macht daraus eine Liste
# und concat macht aus der Liste von DataFrames ein Gesamt-DataFrame
wiktionary_merkmal = pd.concat(  wiktionary.parallel_apply(extract_all, plan=plan, axis=1).tolist()  )

ERROR1 2ERROR
time: 7h 3min 32s


In [30]:
# nachbearbeiten
wiktionary_merkmal = pak.reset_index( wiktionary_merkmal )
wiktionary_merkmal = pak.move_cols(   wiktionary_merkmal, ['section_id','section_id2','merkmal'])
wiktionary_merkmal = pak.drop_cols(   wiktionary_merkmal, ['node_link'])
wiktionary_merkmal = pak.rename_col(  wiktionary_merkmal, 'node_num',  'num')
wiktionary_merkmal = pak.rename_col(  wiktionary_merkmal, 'node_meta', 'meta')
wiktionary_merkmal = pak.rename_col(  wiktionary_merkmal, 'node_text', 'data')    

time: 12.6 s


## Kontrolle: wiktionary_merkmal

In [31]:
# Merkmale Häufigkeit
v = pak.analyse_freqs(wiktionary_merkmal, 'merkmal') 
v

,merkmal,merkmal_count,merkmal_percent,graph
0,gm,2476867,48.8,########################
1,übersicht,1174018,23.1,###########
2,def,271480,5.3,##
3,familie,177932,3.5,#
4,down,167582,3.3,#
5,kombi,144226,2.8,#
6,alt_1,141725,2.8,#
7,syn,139047,2.7,#
8,up,108079,2.1,#
9,lateral,106094,2.1,#


time: 1.89 s


In [32]:
# plan erfüllt?
geplant = set(plan.merkmal)
done    = set(v.merkmal)
nicht_erfüllt = geplant - done
print(nicht_erfüllt)
if version == '_all': 
    assert len(nicht_erfüllt) <= 1  # {'reim'} ist nicht erfüllt

set()
time: 63.5 ms


In [33]:
# Suche festlegen
suche_debug = ['Haus',]

time: 31.2 ms


In [34]:
# Alle Merkmale einer Section
mask = wiktionary.section_id2.isin(suche_debug)
suche_section_ids = wiktionary[mask].section_id.tolist()
mask = wiktionary_merkmal.section_id.isin(suche_section_ids)
a = pak.analyse_freqs( wiktionary_merkmal[mask], ['merkmal','data'], limits=[999,None])
#grid(a)
a

,merkmal,merkmal_count,merkmal_percent,data,data_count
0,kombi,460,36.2,"[ein, das, die, sein, ins, gehen, der, jemande...","[28, 17, 7, 6, 6, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5..."
1,down,391,30.7,"[Hochzeitshaus, Doppelhaus, Kartenhaus, Apartm...","[3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, ..."
2,familie,275,21.6,"[hausschlachten, aushäusig, dreihäusig, einhäu...","[2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,lateral,49,3.9,"[Spelunke, Bleibe, Klause, Logis, Quartier, Un...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,übersicht,38,3.0,"[mini, Haus, Häuser, 2, n, Hauses, Hause, Häus...","[7, 3, 3, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
5,up,18,1.4,"[Ort, Bau, Gebäude, Personengruppe, Personenkr...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
6,syn,16,1.3,"[Winde, Affenkasten, Kasten, Wohnhaus, Wohngeb...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
7,herkunft,8,0.6,"[hūs, huus, hús]","[6, 1, 1]"
8,klein,7,0.6,"[Häuschen, Häuserchen, Häusli, Häusl, Häusel, ...","[1, 1, 1, 1, 1, 1, 1]"
9,def,5,0.4,"[Gebäude, Weichtier, Gastropode]","[3, 1, 1]"


time: 1.42 s


In [35]:
# Alle Merkmale einer Section
mask = wiktionary.title.isin(suche_debug)
suche_section_ids = wiktionary[mask].section_id.tolist()
mask = wiktionary_merkmal.section_id.isin(suche_section_ids)
#grid(wiktionary_merkmal, mask)
wiktionary_merkmal[mask]

,section_id,section_id2,merkmal,node_debug,node_kontext,data,num,meta
10611,21600,Haus,übersicht,Deutsch Substantiv Übersicht\n,Genus,n,NaN,NaN
10612,21600,Haus,übersicht,Deutsch Substantiv Übersicht\n,Nominativ Singular,Haus,NaN,NaN
10613,21600,Haus,übersicht,Deutsch Substantiv Übersicht\n,Nominativ Plural,Häuser,NaN,NaN
10614,21600,Haus,übersicht,Deutsch Substantiv Übersicht\n,Genitiv Singular,Hauses,NaN,NaN
10615,21600,Haus,übersicht,Deutsch Substantiv Übersicht\n,Genitiv Plural,Häuser,NaN,NaN
...,...,...,...,...,...,...,...,...
11880,21600,Haus,familie,,,haushalten,[],Adjektive
11881,21600,Haus,familie,,,hausieren,[],Adjektive
11882,21600,Haus,familie,,,hausschlachten,[],Adjektive
11883,21601,Haus_1,alt_1,,,Haues,[],


time: 293 ms


In [36]:
# Ein Merkmal in allen Sections
mask = (wiktionary_merkmal.merkmal == 'up_part')
#grid(wiktionary_merkmal, mask)
wiktionary_merkmal[mask]

,section_id,section_id2,merkmal,node_debug,node_kontext,data,num,meta
12627,22400,Wiktionary,up_part,,,Wikimedia,[1],
21834,35701,Lenzing_1,up_part,,,Vöcklabruck,[1],
21835,35701,Lenzing_1,up_part,,,Oberösterreich,[1],
27049,46400,Tag,up_part,,,Woche,[2],
27050,46400,Tag,up_part,,,Monat,[2],
...,...,...,...,...,...,...,...,...
4924040,104887700,Habel,up_part,,,Nordfriesland,[1],
4924049,104887900,Oland,up_part,,,Nordfriesland,[1],
4927777,104960300,Harmisch,up_part,,,Kohfidisch,[1],
4964030,105605100,Deutsch_Tschantschendorf,up_part,,,Burgenland,[1],


time: 293 ms


## Müll löschen

In [37]:
# Reste von Bildern
mask1 = (wiktionary_merkmal.merkmal == 'übersicht')
mask2 = (wiktionary_merkmal.node_kontext.str.len() < 3)
mask = mask1  &  mask2
wiktionary_merkmal = pak.drop_rows(wiktionary_merkmal,mask, verbose=True)

Delete 99788 rows from 5075879
time: 2.92 s


In [38]:
# wiktionary_merkmal: Spalten löschen
wiktionary_merkmal = pak.drop_cols(wiktionary_merkmal,['section_id'])

time: 1.37 s


## Speichern

In [39]:
# Speichern
if wiktionary_merkmal_filename:
    pak.dump_pickle(wiktionary_merkmal, wiktionary_merkmal_filename) 

time: 28.8 s


In [40]:
# Ausgabeverzeichnis touch
import os
os.utime(verzeichnis_save)

time: 34.6 ms


In [105]:
# Save in Google Drive
if environment['in_colab']:
    from google.colab import drive
    drive.mount('/content/drive') 
    !cd /content
    !cp -av {verzeichnis_save} drive/MyDrive/{verzeichnis_save}  

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
'data_00' -> 'drive/MyDrive/data_00/data_00'
'data_00/dewiktionary-latest-pages-articles-multistream.xml.bz2' -> 'drive/MyDrive/data_00/data_00/dewiktionary-latest-pages-articles-multistream.xml.bz2'
'data_00/wiktionary_short.pickle' -> 'drive/MyDrive/data_00/data_00/wiktionary_short.pickle'
'data_00/wiktionary_trash_short.pickle' -> 'drive/MyDrive/data_00/data_00/wiktionary_trash_short.pickle'
'data_00/wiktionary_all.pickle' -> 'drive/MyDrive/data_00/data_00/wiktionary_all.pickle'
'data_00/wiktionary_trash_all.pickle' -> 'drive/MyDrive/data_00/data_00/wiktionary_trash_all.pickle'
time: 6.9 s (started: 2023-02-15 16:36:47 +00:00)


In [41]:
# memory_consumption
pak.memory_consumption( locals() )

,name,rtype,size
0,wiktionary,"(DataFrame, Series)",2.8 GB
1,wiktionary_merkmal,"(DataFrame, Series)",2.2 GB
2,wiktionary_trash,"(DataFrame, Series)",15.3 MB
3,mask,"(Series, bool)",4.8 MB
4,mask1,"(Series, bool)",4.8 MB
5,mask2,"(Series, bool)",4.8 MB
6,__,"(DataFrame, Series)",577.7 KB
7,_35,"(DataFrame, Series)",577.7 KB
8,testzeile,"(Series, int32)",344.8 KB
9,_,"(DataFrame, Series)",199.9 KB


time: 37.6 s
